### MAIN: Data Cleaner
#### Choose clean non-pooled data only

In [5]:
import json
import pandas as pd

# Function to convert time in seconds to floating-point seconds
def convert_to_seconds(time, unit):
    if unit == "seconds":
        return float(time)
    elif unit == "minutes":
        return float(time) * 60  # Convert minutes to seconds

# Step 1: Load the data from "new.json" with multiple JSON objects separated by new lines
with open("new.json", "r") as json_file:
    new_data = [json.loads(line) for line in json_file]  # Read each line as a JSON object

# Step 2: Load the data from "pooling.csv"
pooling_data = pd.read_csv("anon_pooling_jan_24_amman.csv")

# Step 3: Filter pooling_data to only include records with the 'day' attribute set to '2024-01-31'
filtered_pooling_data = pooling_data[pooling_data['day'] == '2024-01-31']

output_file = "pooling_by_day.csv"
filtered_pooling_data.to_csv(output_file, index=False)
# with open("pooling_by_day.csv", "w") as output_file:
    # json.dump(filtered_pooling_data, output_file)


Matching process completed. Check 'clean_trips.json' for results.


In [9]:
import pandas as pd

input_file = "pooling_by_day.csv"
filtered_pooling_data = pd.read_csv(input_file)

# Step 1: Load the data from "new.json" with multiple JSON objects separated by new lines
with open("new.json", "r") as json_file:
    new_data = [json.loads(line) for line in json_file]  # Read each line as a JSON object

output_file = "clean_trips.json"

# Step 3: Initialize an empty list to store matched records
# matched_records = []

with open(output_file_path, 'w', encoding='utf-8') as output_file:
# Step 4: Iterate over the data from "new.json"
    for entry in new_data:
        entry_time = entry["time"]
        entry_trip_id = entry["trip_id"]
        entry_driver_id = entry["driverID"]

        # Step 5: Search for a matching entry in "pooling.csv"
        for _, row in filtered_pooling_data.iterrows():
            pool_time = float(row["captain_engagement_time"] * 60)
            pool_trip_id = row["booking_id"]
            pool_driver_id = row["captain_id"]

            # Step 6: Check for trip_id, driver_id, and time margin of 5 seconds
            if (entry_trip_id == pool_trip_id and
                entry_driver_id == pool_driver_id and
                abs(pool_time - entry_time) <= 5):
                # If match found, add to matched_records
                # matched_records.append(entry)
                # with open(output_file_path, 'w', encoding='utf-8') as output_file:
                #     output_file.write(entry)
                json.dump(entry, output_file)
                output_file.write("\n")
                break

# Step 7: Write the matched records to "clean_trips.json"
# with open("clean_trips.json", "w") as output_file:
#     json.dump(matched_records, output_file)

print("Matching process completed. Check 'clean_trips.json' for results.")

Matching process completed. Check 'clean_trips.json' for results.
